# Building the Streaming Dataflow Accelerator

<font color="red">**Live FINN tutorial:** 이 노트북을 읽기 시작할 때 **Cell -> Run All**을 클릭하는 것을 권장합니다.
이를 통해 지연(latency) 숨김 효과를 얻을 수 있습니다.</font>

**중요: 이 노트북은 1-train-mlp-with-brevitas 노트북에 의존합니다.
해당 노트북에서 생성된 모델 파일(.onnx) 을 사용하고 있기 때문입니다.
따라서, 이 노트북을 실행하기 전에 필요한 .onnx 파일들이 먼저 생성되어 있는지 확인해 주세요.**

<img align="left" src="../finn-example.png" alt="drawing" style="margin-right: 20px" width="250"/>

이 노트북에서는 FINN 컴파일러를 사용하여, 사이버 보안 작업을 위한 양자화된 MLP 모델로부터 Streaming Dataflow 아키텍처 기반의 FPGA 가속기를 생성할 것입니다.
이 아키텍처의 핵심 아이디어는, 레이어 간 병렬화뿐만 아니라 각 레이어 내에서도 병렬화하여, 각 레이어에 비례한 연산 자원을 할당하는 것입니다. (왼쪽 그림에 나타난 구조)
이 개념에 대한 자세한 내용은 FINN 논문과 FINN-R 논문에서 확인할 수 있습니다.

이를 위해 각 레이어를 Vivado HLS (High-Level Synthesis)로 변환하고, 각 레이어가 요구하는 병렬화 수준에 맞춰 구현한 뒤, On-Chip FIFO를 사용해 레이어들을 연결하여 전체 가속기를 구성합니다.

이러한 방식은 성능과 유연성의 균형을 제공하지만, 수작업으로 구현하기에는 매우 어렵고 시간이 많이 소요됩니다.
그래서 FINN 컴파일러가 필요한 것입니다.
FINN 컴파일러는 ONNX 포맷의 모델을 입력받아, 원하는 처리량(Throughput)에 맞는 Streaming Dataflow 가속기를 자동으로 생성해줍니다.

## Outline
-------------

1. [Introduction to  `build_dataflow` Tool](#intro_build_dataflow) 
2. [Understanding the Build Configuration: `DataflowBuildConfig`](#underst_build_conf)     
    2.1.[Output Products](#output_prod)   
    2.2.[Configuring the Board and FPGA Part](#config_fpga)   
    2.3 [Configuring the Performance](#config_perf)    
4. [Launch a Build: Only Estimate Reports](#build_estimate_report)
5. [Launch a Build: Stitched IP, out-of-context synth and rtlsim Performance](#build_ip_synth_rtlsim)
6. [(Optional) Launch a Build: PYNQ Bitfile and Driver](#build_bitfile_driver)
7. [(Optional) Run on PYNQ board](#run_on_pynq)

## Introduction to  `build_dataflow` Tool <a id="intro_build_dataflow"></a>

버전 0.5b부터 FINN 컴파일러는 `build_dataflow` 도구를 제공한다. 이전 버전에서는 모든 필요한 변환 과정을 Python 스크립트로 직접 설정해야 했던 것과 달리, 이 도구를 통해 데이터플로우 아키텍처 생성을 보다 쉽게 실험할 수 있다. 핵심 개념은 관련 빌드 정보를 설정한 구성 `dict`를 제공하면, 필요한 모든 단계가 자동으로 수행되어 데이터플로우 빌드가 진행된다는 것이다. 이 도구는 [command line](https://finn-dev.readthedocs.io/en/latest/command_line.html) 에서 실행하거나, Python 함수 한 줄로 호출할 수 있다.


이 노트북에서는 Jupyter 노트북 내에서 작업을 계속하기 위해 Python 함수 호출 방식을 사용하여 빌드를 실행할 것이다. 그러나 여기서 수행하는 작업을 `./run-docker.sh build_dataflow` 또는 `./run-docker.sh build_custom` 명령어를 사용해 명령줄에서 재현해보는 것도 자유롭게 시도해도 좋다.

build_dataflow 도구 소개 <a id="intro_build_dataflow"></a>
버전 0.5b부터, FINN 컴파일러는 build_dataflow 도구를 제공합니다.
이전 버전에서는 Python 스크립트 내에서 필요한 모든 변환을 직접 설정해야 했던 것과 달리,
이 도구를 사용하면 Dataflow 아키텍처 생성을 보다 쉽게 실험할 수 있습니다.

핵심 아이디어는, 필요한 빌드 정보를 하나의 구성 dict로 지정하면,
Dataflow 빌드를 수행하는 데 필요한 모든 단계를 자동으로 실행해준다는 것입니다.
이 도구는 커맨드 라인에서 실행할 수도 있고,
단일 Python 함수 호출로도 실행할 수 있습니다.

이 노트북에서는 Jupyter Notebook 환경을 유지하기 위해 Python 함수 호출 방식으로 빌드를 진행하겠지만,
원한다면 여기서 진행하는 작업을
./run-docker.sh build_dataflow 와 ./run-docker.sh build_custom 커맨드라인 명령어로도 시도해볼 수 있습니다.

## 빌드 설정 이해하기: `DataflowBuildConfig` <a id="underst_build_conf"></a>

빌드 설정은`finn.builder.build_dataflow_config.DataflowBuildConfig` 인스턴스를 통해 지정한다. 이 설정은 Python의 [`dataclass`](https://docs.python.org/3/library/dataclasses.html) 로 정의되어 있으며, JSON 파일로 직렬화하거나 역직렬화하여 저장할 수도 있다. 그러나 여기서는 Python 코드에서 직접 설정할 것이다.
설정에는 빌드의 다양한 측면을 커스터마이즈할 수 있는 많은 옵션들이 포함되어 있지만, 이 노트북에서는 그중 일부만 다룰 것이다. 모든 설정 옵션에 대한 자세한 내용은 [FINN API 문서](https://finn-dev.readthedocs.io/en/latest/source_code/finn.builder.html#finn.builder.build_dataflow_config.DataflowBuildConfig).에서 확인할 수 있다.

`DataflowBuildConfig`의 일부 구성 요소를 살펴보자:

### 출력물 <a id="output_prod"></a>

빌드 과정에서는 다양한 출력물을 생성할 수 있으며, 일부 출력물은 시간이 오래 걸릴 수 있다 (예: 대규모 네트워크에 대한 비트파일 합성). 새로운 가속기 생성 작업을 시작하고 다양한 성능 옵션을 탐색할 때는 처음부터 비트파일까지 생성할 필요가 없을 수도 있다. 따라서 초반에는 추정 보고서(Estimate Reports)만 출력물로 선택하는 것이 일반적이다. 이후 설계가 만족스러워질 때까지 점진적으로 다음 단계의 출력물을 생성하고, 최종적으로는 쉘에 통합된 전체 가속기까지 빌드할 수 있다.

The output products are controlled by:

* `generate_outputs`: 빌드 과정에서 생성할 출력물의 목록. 각각의 출력물은[`finn.builder.build_dataflow_config.DataflowOutputType`](https://finn-dev.readthedocs.io/en/latest/source_code/finn.builder.html#finn.builder.build_dataflow_config.DataflowOutputType)) 타입이다. 사용 가능한 옵션 일부는 다음과 같다:
    - `ESTIMATE_REPORTS` : 합성 없이, 각 레이어 및 전체 네트워크에 대한 예상 리소스 사용량과 성능을 보고서 형태로 제공
    - `STITCHED_IP` : 다른 Vivado IPI 또는 RTL 디자인에 통합할 수 있는 스트림 입력-출력 IP 디자인 생성
    - `RTLSIM_PERFORMANCE` : PyVerilator를 사용하여 STITCHED_IP 디자인의 성능 및 지연 시간 테스트 수행
    - `OOC_SYNTH` : 주변 시스템 없이 가속기 자체만을 대상으로 Out-of-Context 합성 수행하여 합성 이후의 FPGA 리소스 사용량 및 달성 가능한 클럭 주기 확인
    - `BITFILE` : 가속기를 쉘에 통합하여 독립 실행형 비트파일 생성
    - `PYNQ_DRIVER` : 가속기 실행에 사용할 수 있는 PYNQ Python 드라이버 생성
    - `DEPLOYMENT_PACKAGE` : `BITFILE`과 `PYNQ_DRIVER`를 포함하는 폴더 생성, 대상 FPGA 플랫폼에 복사하여 바로 사용할 수 있도록 구성
* `output_dir`: 위의 빌드 출력물들이 저장될 디렉터리
* `steps`: FINN이 빌드 과정에서 수행할 사전 정의된 (또는 커스텀) 빌드 단계 목록. 추정 단계만 실행하려면`build_dataflow_config.estimate_only_dataflow_steps` 를 사용하고, 그렇지 않으면 기본값인 `build_dataflow_config.default_build_dataflow_steps` 를 사용한다. 기본 빌드 단계 목록은 [여기](https://finn.readthedocs.io/en/latest/source_code/finn.builder.html#finn.builder.build_dataflow_config.default_build_dataflow_steps) 에서 확인할 수 있다.

### 보드 및 FPGA 파트 설정 <a id="config_fpga"></a>

* `fpga_part`: 합성(Synthesis)에 사용할 Xilinx FPGA 파트. 아래의 `board` 설정으로부터 추론할 수 있으며, 특정 상황에서는 명시적으로 지정할 수도 있다 (예: Out-of-Context 합성).
* `board`: 가속기를 쉘(Shell)에 통합하여 생성할 때 사용할 대상 Xilinx Zynq 또는 Alveo 보드. 사용 가능한 보드 목록은 [이 파일](https://github.com/Xilinx/finn-base/blob/dev/src/finn/util/basic.py#L41) 의`pynq_part_map` 및 `alveo_part_map` 딕셔너리를 참고.
* `shell_flow_type`: FINN 디자인을 쉘에 통합하여 전체 비트파일(Bitfile)을 생성할 때 사용하는 [shell flow type](https://finn-dev.readthedocs.io/en/latest/source_code/finn.builder.html#finn.builder.build_dataflow_config.ShellFlowType), `BITFILE` 생성 옵션을 선택한 경우에만 필요하다.

### 성능 설정 <a id="config_perf"></a>

생성된 데이터플로우 가속기의 성능(그리고 이에 따른 FPGA 리소스 사용량)은 두 가지 방법으로 설정할 수 있다:

1) (기본) 목표 성능을 설정하면, 컴파일러가 각 노드별 병렬화 설정을 자동으로 결정한다.

2) (고급) 각 레이어에 대한 병렬화 정도와 기타 하드웨어 옵션을 명시한 별도의 .json 파일을 `folding_config_file`로 지정한다.

이 노트북에서는 기본 방법만 다루며, 이를 위해 다음과 같은 설정이 필요하다:

* `target_fps`: 초당 프레임 수(frames per second) 기준의 목표 추론 성능. 특정 레이어 제약이나 FPGA 자원 한계로 인해 설정한 목표 성능이 달성되지 않을 수 있다.
* synth_clk_period_ns: Vivado 합성을 위한 목표 클럭 주기(ns 단위). 예를 들어, synth_clk_period_ns=5.0으로 설정하면 200 MHz 클럭을 목표로 한다. 그러나 FPGA 파트나 설계 복잡도에 따라 해당 클럭 주기가 달성되지 않을 수 있다.

## 빌드 실행: 추정 보고서만 생성 <a id="build_estimate_report"></a>

먼저, 우리는 추정 보고서(Estimate Reports)만 생성하는 빌드를 실행할 것이다. 이 빌드는 합성(Synthesis)을 필요로 하지 않는다. 아래 두 가지에 주목하라: `generate_outputs`가 `ESTIMATE_REPORTS`만 포함하고 있으며, `steps`에서는 `estimate_only_dataflow_steps` 값을 사용하고 있다. 이는 HLS 합성(HLS synthesis)과 같은 단계를 건너뛰고, 분석 모델(analytical models)을 통해 빠른 추정을 제공하기 위함이다.

In [1]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_dir = os.environ['FINN_ROOT'] + "/notebooks/end2end_example/cybersecurity/custom"
# model_file = model_dir + "/cybsec-mlp-ready.onnx"
model_file = model_dir + "/plz_work.onnx"

estimates_output_dir = "plz_output"

# 경로가 존재하면 삭제한다
if os.path.exists(estimates_output_dir):
    shutil.rmtree(estimates_output_dir)
    print("Previous run results deleted!")


cfg_estimates = build.DataflowBuildConfig(
    output_dir          = estimates_output_dir, # 결과 저장 폴더
    mvau_wwidth_max     = 50, #MVAU (행렬-벡터 곱 유닛) 최대 가중치 폭 제한 (자원 제어)
    target_fps          = 5000, #원하는 추론 속도 목표
    synth_clk_period_ns = 5.0, #합성 시 사용될 클럭 주기 (5ns = 200MHz 클럭)
    fpga_part           = "xc7z020clg400-1", # 사용할 FPGA 칩
    steps               = build_cfg.estimate_only_dataflow_steps, # 실제 빌드 하지 않고, 자원 사용량/ 속도 등 추정만 수행
    generate_outputs=[ # 리포트만 생성
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
    ]
)


Previous run results deleted!


In [2]:
%%time
build.build_dataflow_cfg(model_file, cfg_estimates)

Building dataflow accelerator from /home/cloud9/finn/notebooks/end2end_example/cybersecurity/custom/plz_work.onnx
Intermediate outputs will be generated in /tmp/finn_dev_cloud9
Final outputs will be generated in plz_output
Build log is at plz_output/build_dataflow.log
Running step: step_qonnx_to_finn [1/10]
Running step: step_tidy_up [2/10]
Running step: step_streamline [3/10]
Running step: step_convert_to_hw [4/10]
Running step: step_create_dataflow_partition [5/10]
Running step: step_specialize_layers [6/10]
Running step: step_target_fps_parallelization [7/10]
Running step: step_apply_folding_config [8/10]
Running step: step_minimize_bit_width [9/10]
Running step: step_generate_estimate_reports [10/10]
Completed successfully
CPU times: user 924 ms, sys: 1.45 s, total: 2.37 s
Wall time: 585 ms


0

In [3]:
assert os.path.exists(estimates_output_dir + "/report/estimate_network_performance.json")

이제 이번 빌드 과정에서 생성된 출력 결과들을 살펴보겠습니다.
outputs 디렉토리를 확인해보면, 그 안에 생성된 추정 리포트(estimate reports) 가 포함된 하위 폴더가 있는 것을 확인할 수 있습니다.

In [4]:
! ls {estimates_output_dir}

auto_folding_config.json  report
build_dataflow.log	  template_specialize_layers_config.json
intermediate_models	  time_per_step.json


In [5]:
! ls {estimates_output_dir}/report

estimate_layer_config_alternatives.json  estimate_network_performance.json
estimate_layer_cycles.json		 op_and_param_counts.json
estimate_layer_resources.json


여러 개의 리포트가 .json 파일 형식으로 생성된 것을 확인할 수 있습니다.
먼저, `estimate_network_performance.json` 파일의 내용을 살펴보겠습니다.
이 파일에는 성능(Throughput)과 지연 시간(Latency)에 대한 분석 기반 추정치가 포함되어 있습니다.

In [6]:
! cat {estimates_output_dir}/report/estimate_network_performance.json

{
  "critical_path_cycles": 24640,
  "max_cycles": 16384,
  "max_cycles_node_name": "MVAU_hls_0",
  "estimated_throughput_fps": 12207.03125,
  "estimated_latency_ns": 123200.0
}

이러한 리포트들은 모두 .json 파일 형식이기 때문에,
Python에서 쉽게 불러와서 추가적으로 가공하거나 분석할 수 있습니다.
이는 만약 사용자가 FINN 위에 자체적인 자동화 도구(design automation tool)를 구축하려 할 때 유용하게 활용될 수 있습니다.

이제, 이를 위해 헬퍼 함수를 정의하고,
`estimate_layer_cycles.json` 리포트의 내용을 확인해보겠습니다.

In [7]:
import json
def read_json_dict(filename):
    with open(filename, "r") as f:
        ret = json.load(f)
    return ret

In [9]:
read_json_dict(estimates_output_dir + "/report/estimate_layer_cycles.json")python3 filter_Off.py -f restored_image.jpg


SyntaxError: invalid syntax (3307163787.py, line 1)

여기서는 각 레이어가 소요할 것으로 예상되는 클럭 사이클 수를 확인할 수 있습니다.
모든 레이어는 병렬로 실행되기 때문에,
가장 느린 레이어의 실행 시간이 전체 신경망의 처리량(Throughput) 을 결정하게 됩니다.

FINN은 각 레이어가 비슷한 클럭 사이클 수를 가지도록 병렬화를 시도하며,
설정된 `target_fps` (목표 프레임 속도) 를 달성하는 데 필요한 사이클 수보다 더 적게 소요되도록 조정합니다.

또한, 모든 레이어의 추정 사이클 수를 합산하면
전체 네트워크의 추정 지연 시간(Latency) 을 확인할 수 있습니다.

마지막으로, 레이어별 FPGA 자원 사용량(Resource usage) 에 대한 추정치는
`estimate_layer_resources.json` 리포트에서 확인할 수 있습니다.

In [10]:
read_json_dict(estimates_output_dir + "/report/estimate_layer_resources.json")

{'MVAU_hls_0': {'BRAM_18K': 8,
  'BRAM_efficiency': 0.8888888888888888,
  'LUT': 429,
  'URAM': 0,
  'URAM_efficiency': 1,
  'DSP': 0},
 'MVAU_hls_1': {'BRAM_18K': 4,
  'BRAM_efficiency': 0.8888888888888888,
  'LUT': 429,
  'URAM': 0,
  'URAM_efficiency': 1,
  'DSP': 0},
 'MVAU_hls_2': {'BRAM_18K': 1,
  'BRAM_efficiency': 0.027777777777777776,
  'LUT': 428,
  'URAM': 0,
  'URAM_efficiency': 1,
  'DSP': 0},
 'total': {'BRAM_18K': 13.0, 'LUT': 1286.0, 'URAM': 0.0, 'DSP': 0.0}}

이 리포트는 현재 설정이 특정 FPGA에 적합한지 판단할 때 매우 유용합니다.
만약 리포트에서 나타나는 자원 요구량이 사용하려는 FPGA의 한계보다 높은 경우,
`target_fps` 값을 낮춰서 자원 요구량을 줄이는 것을 고려해야 합니다.

**주의할 점은, 이 리포트의 분석 모델은 실제보다 자원 사용량을 과대 추정하는 경향이 있다는 것입니다.
이는 합성(Synthesis) 과정에서 이루어지는 다양한 최적화 효과를
모델이 정확히 반영하지 못하기 때문입니다.**

##### 빌드 실행 단계: Stitched IP, Out-of-Context Synthesis, RTL Simulation 성능 측정 <a id="build_ip_synth_rtlsim"></a>

만족스러운 추정 결과가 나왔다면, 이제 가속기(Accelerator)를 실제로 생성하는 단계로 넘어갈 수 있습니다.
가속기 생성 방식은, 이를 더 큰 시스템에 어떻게 통합할 것인지에 따라 여러 가지로 선택할 수 있습니다.

예를 들어:

Vivado에서 구축한 스트리밍 시스템에 통합하거나

다른 프로젝트에서 재사용 가능한 IP(지적 재산) 컴포넌트로 사용하고자 할 경우
→ STITCHED_IP 출력물을 생성하는 것이 좋은 선택입니다.

또한, 생성된 가속기의 합성 이후(Post-synthesis)의 자원 사용량 및 클럭 주파수 정보를 얻고 싶다면
OOC_SYNTH 출력물을 사용할 수 있습니다.

<font color="red">**Live FINN tutorial:** 다음 빌드 과정은 Vivado를 여러 번 호출하고 RTL 시뮬레이션을 수행하기 때문에 약 10분 정도 소요됩니다.
빌드가 진행되는 동안, noVNC를 통해 생성된 파일들을 확인할 수 있습니다 — noVNC는 현재 실행 중입니다. **(your AWS URL):6080/vnc.html**

* 아래의 `step_hls_codegen [8/16]` 단계가 완료되면, 각 레이어별로 생성된 HLS 코드를 해당 폴더에서 확인할 수 있다: `/tmp/finn_dev_ubuntu/code_gen_ipgen_MVAU_hls_XXXXXX`
    
* 아래의 `step_create_stitched_ip [11/16]` 단계가 완료되면, Vivado에서 생성된 Stitched IP를 다음 경로에서 확인할 수 있다:`/home/ubuntu/finn/notebooks/end2end_example/cybersecurity/output_ipstitch_ooc_rtlsim/stitched_ip`
</font> 

In [14]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_file = model_dir + "/plz_work.onnx"

rtlsim_output_dir = "plz_word_output_ipstitch_ooc_rtlsim"

#Delete previous run results if exist
if os.path.exists(rtlsim_output_dir):
    shutil.rmtree(rtlsim_output_dir)
    print("Previous run results deleted!")

#고급 빌드 설정
cfg_stitched_ip = build.DataflowBuildConfig(
    output_dir          = rtlsim_output_dir, #출력 결과 저장 위치
    mvau_wwidth_max     = 40, # 위와 같음
    target_fps          = 5000, #위와 같음
    synth_clk_period_ns = 5.0, # 위와 같음
    fpga_part           = "xc7z020clg400-1", # 같음
    generate_outputs=[ #어떤 빌드 단계를 수행할지 지정하는 리스트
        #HSL로 생성된 각 계층을 IP로 만들고 IP연결까지 수행
        build_cfg.DataflowOutputType.STITCHED_IP,
        # 실제 RTL레벨 시뮬레이션을 통해 정확한 성능 예측 수행
        build_cfg.DataflowOutputType.RTLSIM_PERFORMANCE,
        # Vivado에서 Out-Of-Context 방식으로 합성 수행 -> LUT, DSP, BRAM 사용량 추정
        build_cfg.DataflowOutputType.OOC_SYNTH,
    ]
)

Previous run results deleted!


In [15]:
%%time
build.build_dataflow_cfg(model_file, cfg_stitched_ip)
print("Success")

Building dataflow accelerator from /home/cloud9/finn/notebooks/end2end_example/cybersecurity/custom/plz_work.onnx
Intermediate outputs will be generated in /tmp/finn_dev_cloud9
Final outputs will be generated in plz_word_output_ipstitch_ooc_rtlsim
Build log is at plz_word_output_ipstitch_ooc_rtlsim/build_dataflow.log
Running step: step_qonnx_to_finn [1/19]
Running step: step_tidy_up [2/19]
Running step: step_streamline [3/19]
Running step: step_convert_to_hw [4/19]
Running step: step_create_dataflow_partition [5/19]
Running step: step_specialize_layers [6/19]
Running step: step_target_fps_parallelization [7/19]
Running step: step_apply_folding_config [8/19]
Running step: step_minimize_bit_width [9/19]
Running step: step_generate_estimate_reports [10/19]
Running step: step_hw_codegen [11/19]
Running step: step_hw_ipgen [12/19]
Running step: step_set_fifo_depths [13/19]
Running step: step_create_stitched_ip [14/19]
Running step: step_measure_rtlsim_performance [15/19]
Running step: step_

In [16]:
assert os.path.exists(rtlsim_output_dir + "/report/ooc_synth_and_timing.json")
assert os.path.exists(rtlsim_output_dir + "/report/rtlsim_performance.json")
assert os.path.exists(rtlsim_output_dir + "/final_hw_config.json")

예를 들어, 우리가 출력물로 bitfile을 요청하지 않았는데도 위에 `step_synthesize_bitfile` 이 나열된 이유는 무엇일까요?
이는 우리가 기본 빌드 단계(default set of build steps) 를 사용하고 있기 때문입니다.
기본 단계에는 `step_synthesize_bitfile` 도 포함되어 있지만,
출력물로 bitfile이 선택되지 않았기 때문에 이 단계는 실제로 아무 작업도 수행하지 않습니다.

출력물들 중에는, Stitched IP 블록 디자인 형태로 내보내진 가속기(Accelerator) 가 포함되어 있습니다.

In [17]:
! ls {rtlsim_output_dir}/stitched_ip

all_verilog_srcs.txt		       finn_vivado_stitch_proj.xpr
data				       ip
finn_vivado_stitch_proj.cache	       make_project.sh
finn_vivado_stitch_proj.gen	       make_project.tcl
finn_vivado_stitch_proj.hw	       vivado.jou
finn_vivado_stitch_proj.ip_user_files  vivado.log
finn_vivado_stitch_proj.srcs


또한, 이번 출력물들에 의해 생성된 몇 가지 리포트(report) 도 있습니다.
이 리포트들은 이전에 생성된 `ESTIMATE_REPORTS` 와는 다른 종류의 리포트입니다.

In [18]:
! ls {rtlsim_output_dir}/report

estimate_layer_resources_hls.json  rtlsim_performance.json
ooc_synth_and_timing.json


`ooc_synth_and_timing.json` 파일에서는 가속기의 합성 이후(Post-synthesis) 정보와 최대 클럭 주파수 추정치를 확인할 수 있습니다.
단, 여기서 제공되는 클럭 주파수 추정치는 제약 조건이 덜 적용된 Out-of-Context 합성 환경에서 나온 것이기 때문에, 다소 낙관적으로(over-estimate) 표시될 수 있습니다.

In [19]:
! cat {rtlsim_output_dir}/report/ooc_synth_and_timing.json

{
  "vivado_proj_folder": "/tmp/finn_dev_cloud9/synth_out_of_context_kqvyvp8m/results_finn_design_wrapper",
  "LUT": 17342.0,
  "LUTRAM": 56.0,
  "FF": 13144.0,
  "DSP": 0.0,
  "BRAM": 140.0,
  "BRAM_18K": 268.0,
  "BRAM_36K": 6.0,
  "URAM": 0.0,
  "Carry": 1600.0,
  "WNS": -2.153,
  "Delay": -2.153,
  "vivado_version": 2024.1,
  "vivado_build_no": 5076996.0,
  "": 0,
  "fmax_mhz": 139.8014818957081,
  "estimated_throughput_fps": 8532.80529148609
}

`rtlsim_performance.json` 파일에서는 RTL 시뮬레이션(rtlsim) 을 통해 얻은 가속기의 안정 상태 처리량(Steady-state throughput) 과 지연 시간(Latency) 을 확인할 수 있습니다.
여기서 보고된 DRAM 대역폭(Bandwidth) 수치가 실제 하드웨어 플랫폼이 제공하는 대역폭보다 낮다면
(즉, 가속기가 메모리 대역폭에 의해 병목되지 않는다면)
실제 하드웨어에서도 동일한 처리량(소프트웨어/드라이버 오버헤드 제외)을 기대할 수 있습니다.

In [20]:
! cat {rtlsim_output_dir}/report/rtlsim_performance.json

{
  "N_IN_TXNS": 128,
  "N_OUT_TXNS": 1,
  "cycles": 24476,
  "N": 1,
  "latency_cycles": 24476,
  "runtime[ms]": 0.12238,
  "throughput[images/s]": 8171.269815329302,
  "fclk[mhz]": 200.0,
  "stable_throughput[images/s]": 8171.269815329302
}

마지막으로, `final_hw_config.json` 파일을 확인해봅시다.
이 파일에는 FINN 컴파일러가 결정한 노드별 하드웨어 구성 정보가 담겨 있습니다.
여기에는 FIFO 깊이(FIFO depths), 병렬화 설정 (PE/SIMD) 등 다양한 설정 값들이 포함되어 있습니다.

추가로 빌드를 더 최적화(Advanced Method) 하고 싶다면,
이 .json 파일을 `folding_config_file` 로 지정하여
새로운 빌드를 시작할 때 출발점으로 활용할 수 있습니다.

In [21]:
! cat {rtlsim_output_dir}/final_hw_config.json

{
  "Defaults": {},
  "StreamingFIFO_rtl_0": {
    "ram_style": "auto",
    "depth": 128,
    "impl_style": "rtl",
    "inFIFODepths": [
      0
    ],
    "outFIFODepths": [
      0
    ]
  },
  "MVAU_hls_0": {
    "PE": 1,
    "SIMD": 1,
    "ram_style": "auto",
    "resType": "auto",
    "mem_mode": "internal_decoupled",
    "runtime_writeable_weights": 0,
    "inFIFODepths": [
      128
    ],
    "outFIFODepths": [
      61
    ]
  },
  "StreamingFIFO_rtl_1": {
    "ram_style": "auto",
    "depth": 61,
    "impl_style": "rtl",
    "inFIFODepths": [
      0
    ],
    "outFIFODepths": [
      0
    ]
  },
  "MVAU_hls_1": {
    "PE": 1,
    "SIMD": 1,
    "ram_style": "auto",
    "resType": "auto",
    "mem_mode": "internal_decoupled",
    "runtime_writeable_weights": 0,
    "inFIFODepths": [
      61
    ],
    "outFIFODepths": [
      2
    ]
  },
  "StreamingFIFO_rtl_2": {
    "ram_style": "auto",
    "depth": 2,
    "impl_style": "rtl",
    "inFIFODepths": [
      0
    ],
    "

## (Optional) 빌드 실행: PYNQ Bitfile 및 드라이버 생성 <a id="build_bitfile_driver"></a>

<font color="red">**Live FINN tutorial:** 이 섹션은 Bitfile 합성 시간(약 15~20분) 으로 인해 핸즈온 튜토리얼에는 포함되어 있지 않습니다.
만약 본인이 PYNQ 보드를 보유하고 있다면,
튜토리얼 이후에 아래 셀들의 주석을 해제하여 직접 실행해볼 것을 권장합니다.</font>

In [3]:

###########################################################
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

# model_file = model_dir + "/cybsec-mlp-ready.onnx"
model_file = model_dir +"/plz_work.onnx"

final_output_dir = "plz_work_output_final"

#Delete previous run results if exist
if os.path.exists(final_output_dir):
    shutil.rmtree(final_output_dir)
    print("Previous run results deleted!")

# 실제 빌드
cfg = build.DataflowBuildConfig(
    output_dir          = final_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 100000,
    synth_clk_period_ns = 10.0,
    board               = "Pynq-Z2",
    # Vivado Zynq용 흐름 선택 (Zynq SoC용 Vivado build)
    shell_flow_type     = build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        #Vivado를 이용해 FPGA용 bitstream (.bit)파일 생성
        build_cfg.DataflowOutputType.BITFILE,
        # PYNQ python에서 사용할 수 있는 드라이버 파일 생성
        # PYNQ에서 Python코드로 모델 실행 가능
        build_cfg.DataflowOutputType.PYNQ_DRIVER,
        # 위의 파일들을 하나의 배포용 압축 패키지로 정리
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ]
)
print("??")


Previous run results deleted!
??


In [4]:
%%time
build.build_dataflow_cfg(model_file, cfg)

Building dataflow accelerator from /home/cloud9/finn/notebooks/end2end_example/cybersecurity/custom/plz_work.onnx
Intermediate outputs will be generated in /tmp/finn_dev_cloud9
Final outputs will be generated in plz_work_output_final
Build log is at plz_work_output_final/build_dataflow.log
Running step: step_qonnx_to_finn [1/19]
Running step: step_tidy_up [2/19]
Running step: step_streamline [3/19]
Running step: step_convert_to_hw [4/19]
Running step: step_create_dataflow_partition [5/19]
Running step: step_specialize_layers [6/19]
Running step: step_target_fps_parallelization [7/19]
Running step: step_apply_folding_config [8/19]
Running step: step_minimize_bit_width [9/19]
Running step: step_generate_estimate_reports [10/19]
Running step: step_hw_codegen [11/19]
Running step: step_hw_ipgen [12/19]
Running step: step_set_fifo_depths [13/19]
Running step: step_create_stitched_ip [14/19]
Running step: step_measure_rtlsim_performance [15/19]
Running step: step_out_of_context_synthesis [16

0

최종 빌드에서는 출력물로 Bitfile과 함께,
Zynq 플랫폼의 PYNQ에서 올바르게 실행하기 위해 필요한 `.hwh` 파일도 포함되어 있습니다.

In [5]:
! ls {final_output_dir}/bitfile

finn-accel.bit	finn-accel.hwh


생성된 Python 드라이버를 사용하면, PYNQ 플랫폼에서 단순히 NumPy 입력/출력으로 가속기를 실행할 수 있습니다.
실행 시 FINN으로 생성된 가속기를 어떻게 사용하는지에 대한 예제 노트북들은 [finn-examples](https://github.com/Xilinx/finn-examples) 저장소에서 확인할 수 있습니다.

In [6]:
! ls {final_output_dir}/driver

driver_base.py	driver.py  finn  qonnx	runtime_weights  validate.py


reports 폴더에는 합성 이후(Post-synthesis)의 자원 사용량(Resource)과 타이밍(Timing) 리포트가 포함되어 있습니다.

In [7]:
! ls {final_output_dir}/report

estimate_layer_resources_hls.json  post_synth_resources.json
post_route_timing.rpt		   post_synth_resources.xml


마지막으로, `deploy` 폴더에는 가속기를 실행하기 위해 대상 보드에 복사해야 할 모든 파일이 포함되어 있습니다.

In [8]:
! ls {final_output_dir}/deploy

bitfile  driver


## (Optional) Run on PYNQ board <a id="run_on_pynq"></a>

<font color="red">**Live FINN tutorial:** 이 섹션은 이전 섹션에서의 Bitfile 합성 시간(약 15~20분) 으로 인해 핸즈온 튜토리얼에는 포함되어 있지 않습니다.
만약 본인이 PYNQ 보드를 보유하고 있다면,
튜토리얼 이후에 아래 셀들의 주석을 해제하여 직접 실행해볼 것을 권장합니다.</font>

보드에서 가속기를 테스트하기 위해,
데이터셋의 사본과 정확도 검증을 위한 Python 스크립트를 `driver` 폴더에 복사한 뒤,
전체 배포 폴더(deploy 폴더) 를 ZIP 아카이브로 압축합니다.

In [10]:
! cp unsw_nb15_binarized.npz {final_output_dir}/deploy/driver

In [11]:
! cp validate-unsw-nb15.py {final_output_dir}/deploy/driver

In [28]:
#! ls {final_output_dir}/deploy/driver

In [12]:
from shutil import make_archive
make_archive('deploy-on-pynq', 'zip', final_output_dir+"/deploy")

'/home/cloud9/finn/notebooks/end2end_example/cybersecurity/custom/deploy-on-pynq.zip'

이제 생성된 zip 파일을 다운로드할 수 있습니다 (**File -> Open**에서 `deploy-on-pynq.zip` 파일 옆의 체크박스를 선택한 후 툴바에서 Download를 클릭), 그런 다음 해당 파일을 PYNQ 보드로 복사하세요 (예: `scp` 또는` rsync` 사용).
그 후, PYNQ 보드에서 다음 명령어를 실행하여 아카이브를 추출하고 검증을 수행하세요:

```shell
unzip deploy-on-pynq.zip -d finn-cybsec-mlp-demo
cd finn-cybsec-mlp-demo/driver
sudo python3.6 -m pip install bitstring
sudo python3.6 validate-unsw-nb15.py --batchsize 1000
```

마지막에 `Final accuracy: 91.868293` 이 출력되는 것을 확인할 수 있습니다.
또한, 검증 과정에서 초당 100만 번의 추론 속도가 완전히 달성되지 않는 것을 눈치챘을 수도 있습니다.
이는 Python의 데이터 패킹/언패킹 처리 및 데이터 이동 오버헤드 때문입니다.

더 자세히 확인하고 싶다면, 생성된 드라이버에 포함된 벤치마킹 모드를 사용하면
실행 시간의 세부 분해(runtime breakdown) 를 확인할 수 있습니다.

```shell
sudo python3.6 driver.py --exec_mode throughput_test --bitfile ../bitfile/finn-accel.bit --batchsize 1000
cat nw_metrics.txt
```

```{'runtime[ms]': 1.0602474212646484,
 'throughput[images/s]': 943176.0737575893,
 'DRAM_in_bandwidth[Mb/s]': 70.7382055318192,
 'DRAM_out_bandwidth[Mb/s]': 0.9431760737575894,
 'fclk[mhz]': 100.0,
 'batch_size': 1000,
 'fold_input[ms]': 9.679794311523438e-05,
 'pack_input[ms]': 0.060115814208984375,
 'copy_input_data_to_device[ms]': 0.002428770065307617,
 'copy_output_data_from_device[ms]': 0.0005249977111816406,
 'unpack_output[ms]': 0.3773000240325928,
 'unfold_output[ms]': 6.818771362304688e-05}```

여기서 여러 번 나타나는 `pack_input/unpack_output` 호출은
입출력 데이터를 numpy 배열에서 가속기가 기대하는 비트 연속 데이터 형식으로 변환할 때 발생하는 패킹/언패킹 오버헤드를 보여줍니다.
또한, `copy_input_data_to_device` 와 `copy_output_data_from_device` 는
데이터를 CPU와 가속기 메모리 사이에서 이동시키는 데 소요되는 비용을 나타냅니다.

이러한 오버헤드는 배치 크기(batch size)가 작은 경우 실행 시간의 대부분을 차지할 수 있습니다.

마지막으로, `throughput[images/s]` 값은
소프트웨어 및 데이터 이동 오버헤드 없이 순수한 하드웨어 처리량을 나타내며,
초당 약 100만 번의 추론(1M inferences per second) 에 근접한 것을 확인할 수 있습니다.